## Note

The other parts of the framework run using command lines, however this part is provided as a Jupyter Notebook. The reason is that the training part was moved to Google Colab due to intensive resources needed to fine tune LLMs. ON Colab, it is easier to work with notebooks than command lines.

# T5

In [ ]:
# import the required classes

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

In [ ]:
# Feel free to set your own values.

model = 'google/flan-t5-small' # the model to be finetuned
trained_model_name = 'flan-t5-small-raw' # the name of the trained model
output_dir = './models' #The output directory where checkpoints will be written.

In [ ]:
# warm up: nr of steps where the lr starts from 0 and goes to initial set, to prevent oferfitting on early data
training_args = Seq2SeqTrainingArguments(output_dir = output_dir,
                                    auto_find_batch_size = True, # Find a correct bvatch size that fits the size of Data.
                                    per_device_train_batch_size=8, # set to 2 if ICEWS14
                                    #per_device_eval_batch_size = 1,
                                    learning_rate = 3e-4, # Higher learning rate than full fine-tuning.
                                    num_train_epochs =  1, # set to 3 if ICEWS14
                                    lr_scheduler_type = 'cosine', # It’s designed to lower the learning rate more gradually than step or exponential decay schedulers, and it often includes a restart mechanism, where the learning rate resets to its initial value at regular intervals before starting the next cycle of decay. This restart helps the model escape from potential local minima by periodically taking larger steps, enabling it to search more thoroughly across the loss landscape.
                                    #eval_strategy="steps",
                                    warmup_steps = 1000,
                                    logging_steps = 5000,
                                    #gradient_accumulation_steps=8,
                                    bf16 = True,
                                    weight_decay = 0.1, # penalizes weights to reduce overfitting
                                    report_to = 'none') # the list of integrations to report the results and logs to.

In [ ]:
# load the model and its tokenizer

from transformers import T5Tokenizer, T5ForConditionalGeneration

training_model = T5ForConditionalGeneration.from_pretrained(model, trust_remote_code = True,
                                                            device_map = 'auto',
                                                            )
tokenizer = T5Tokenizer.from_pretrained(model)

In [ ]:
# set the model to train mode
training_model.train()

In [ ]:
# load a dataset from the data files 
# default path: /data/processed_new/{dataset}/{history_modeling_algorithm}/... From here, choose the train split/document
# IMPORTANT: for ICEWS18, there are two training folders: 1024 and full. Use 1024 for finetuning LLaMA2-7B and full for Flan-T5-Small

from datasets import load_dataset

dataset = load_dataset('json', data_files='./data/processed_new/icews18/raw/icews18_raw_train.json', split = 'train')

In [ ]:
dataset

In [ ]:
# tokenize the input prompt and target entity (as labels)

def process_function(examples):

  inputs = tokenizer(examples['context'], return_special_tokens_mask=True)
  labels = tokenizer(examples['target'], return_special_tokens_mask=True)
  inputs['labels'] = labels['input_ids']

  return inputs


In [ ]:
tokenized_input = dataset.map(process_function, batched = True, remove_columns=dataset.column_names)

In [ ]:
# filter out any input prompt that is longer than the models context size

tokenized_input = tokenized_input.filter(lambda x: len(x['input_ids']) < tokenizer.max_len_single_sentence)

In [ ]:
tokenized_input

In [ ]:
trainer = Seq2SeqTrainer(model = training_model, # We pass in the PEFT version of the foundation model or the standard one if full finetuning is desired
                args = training_args, #The args for the training.
                train_dataset = tokenized_input, #The dataset used to to train the model.
                data_collator=DataCollatorForSeq2Seq(tokenizer, model = training_model), # mlm=False indicates not to use masked language modeling
                )

In [ ]:
trainer.train()

# Llama

In [ ]:
# import the required classes, feel free to add any other PeftConfig type

from transformers import TrainingArguments, Trainer, AutoModelForCausalLM, DataCollatorForLanguageModeling, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
# Feel free to set your own values.

model = 'TheBloke/Llama-2-7B-fp16' # the model to be finetuned
trained_model_name = 'Llama-2-7B-ragtkgc' # the name of the trained model
output_dir = './models' #The output directory where checkpoints will be written.

In [ ]:
# feel free to add any other parameter

lora_config = LoraConfig(r = 8, #As bigger the R bigger the parameters to train.
                        lora_alpha=16, # a scaling factor that adjusts the magnitude of the weight matrix.
                        lora_dropout=0.05, #Helps to avoid Overfitting.
                        bias='lora_only', # this specifies if the bias parameter should be trained.
                        task_type='CAUSAL_LM',
                        use_rslora=True)

In [ ]:
# warm up: nr of steps where the lr starts from 0 and goes to initial set, to prevent oferfitting on early data
training_args = TrainingArguments(output_dir = output_dir,
                                    auto_find_batch_size = True, # Find a correct bvatch size that fits the size of Data.
                                    per_device_train_batch_size=1,
                                    #per_device_eval_batch_size = 1,
                                    learning_rate = 3e-4, # Higher learning rate than full fine-tuning.
                                    num_train_epochs = 1,                                   
                                    warmup_steps = 20,
                                    logging_steps  = 20,
                                    gradient_accumulation_steps=8,
                                    #weight_decay = 0.1, # penalizes weights to reduce overfitting
                                    report_to = 'none') # the list of integrations to report the results and logs to.

In [ ]:
# load the model in a 4bit configuration to use QLora; this is how we trained the models, but you can also load them in 8bit or full if enough resources are available

bnb4_config =  BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4', # precision of the stored weights
                              bnb_4bit_compute_dtype='bfloat16', # precision of computations
                              bnb_4bit_use_double_quant=True
                              )

In [ ]:
# set your own quantization_config if desired

training_model = AutoModelForCausalLM.from_pretrained(model,
                                                         trust_remote_code = True,
                                                         quantization_config = bnb4_config
                                                         ) # for training, device_map does not have to be set. check -> https://huggingface.co/docs/transformers/v4.35.0/main_classes/quantization#bitsandbytes-integration
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# If quantization is applied, enable the line, else disable it.
training_model = prepare_model_for_kbit_training(training_model) # This method wraps the entire protocol for preparing a model before running a training.

In [ ]:
# If PEFT is desired, then get the peft version of the model, else disable it.
training_model = get_peft_model(training_model, lora_config, low_cpu_mem_usage = False) # feel free to put any config file from above. low_cpu_mem_usage — Create empty adapter weights on meta device. Useful to speed up the loading process. Leave this setting as False if you intend on training the model -> https://huggingface.co/docs/peft/package_reference/peft_model

print(training_model.print_trainable_parameters())

In [ ]:
# load a dataset from the data files 
# default path: /data/processed_new/{dataset}/{history_modeling_algorithm}/... From here, choose the train split/document
# IMPORTANT: for ICEWS18, there are two training folders: 1024 and full. Use 1024 for finetuning LLaMA2-7B and full for Flan-T5-Small

dataset = load_dataset('json', data_files='./data/processed_new/icews18/ragtkgc/train/1024/history_modeling_train/icews18.json', split = 'train')

In [ ]:
dataset

In [ ]:
# tokenize the input prompt

def process_function(examples):

  texts = [f"{context} {target}] </s>" for context, target in zip(examples['context'], examples['target'])]

  inputs = tokenizer(texts, return_special_tokens_mask=True)


  return inputs


In [ ]:
tokenized_input = dataset.map(process_function, batched = True, remove_columns=dataset.column_names)

In [ ]:
# filter out any input prompts longer than the model's context size; for LLaMA2-7B, the context size is 4096.
tokenized_input = tokenized_input.filter(lambda x: len(x['input_ids']) < 4095)

In [ ]:
tokenized_input

In [ ]:
# we provide a customized DataCollatorForCompletionLM, to enable the model to only focus on the target entity, not the whole input text, while training
# To train the model to predict only the response and ignore the prompt tokens,
# it sets the label values before the response token to -100.
# This ensures that those tokens are ignored by the PyTorch loss function during training.

# !!! IMPORTANT !!!
# if dataset is icews14, set this to True

is14 = False

from typing import Any, Dict, List, Union
import numpy as np

class DataCollatorForCompletionLM(DataCollatorForLanguageModeling):

    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:

        # The torch_call method overrides the same method in the base class and
        # takes a list of examples as input.
        batch = super().torch_call(examples)

        labels = batch["labels"].clone()

        for i in range(len(examples)):

            # decode the labels tensor to get the original input prompt (for generative models, the labels are the inputs shifted by one to the right)
            # split the prompt to reach the target quadruple

            prompt = tokenizer.decode(batch['labels'][i][:-1])
            quad = prompt.split('\n')[-1].split(' ')[-1]
            quad = quad.strip()

            # remove any tokens that are not part of the target quad
            if quad.endswith(']'):
              quad = quad.replace(']','')
            if '<s> [INST] ' in quad:
              quad = quad.replace('<s> [INST] ','')
            if ' [/INST]' in quad:
              quad = quad.replace('[/INST]','')
            if '\n' in quad:
              quad = quad.replace('\n', '')

            # encode the target quad
            response_token_ids = self.tokenizer.encode(quad)[1:]

            # find where the target quad starts in the prompt and save its index
            response_token_ids_start_idx = None
            response_token_ids_start_idx = np.where(batch["labels"][i] == response_token_ids[0])[0][-1]
            
            if response_token_ids_start_idx is None:
                # If the response token is not found in the sequence, it raises a RuntimeError.
                
                raise RuntimeError(
                    f'Could not find response key {response_token_ids} in token IDs {batch["labels"][i]}')


            response_token_ids_end_idx = response_token_ids_start_idx

            # To train the model to predict only the response and ignore the prompt tokens,
            # it sets the label values before the response token to -100.
            # This ensures that those tokens are ignored by the PyTorch loss function during training.

            # set all labels for the input prompt until the target quad to -100
            labels[i, :response_token_ids_end_idx] = -100

            # there are a few tokens that need to be converted to another ones in order to correctly set the right label for the target quad
            # for example, the target quad may have an object like "Men_(Australia)"; the input prompt history (including the target quad) ends with "]", the tokenizer might encode ")]" together
            # and we are left with "Men_(Australia"; therefore, we change ")]" to ")", as "]" would have had the label -100 anyway and be ignored.

            if labels[i, len(response_token_ids)+response_token_ids_start_idx-1] == 4638: # ")]"
              labels[i, len(response_token_ids)+response_token_ids_start_idx-1] = 29897 # ")"

            if labels[i, len(response_token_ids)+response_token_ids_start_idx-1] == 28166: # "))]"
              labels[i, len(response_token_ids)+response_token_ids_start_idx-1] = 876 # "))"

            if labels[i, len(response_token_ids)+response_token_ids_start_idx-1] == 5586: # ".]"
              labels[i, len(response_token_ids)+response_token_ids_start_idx-1] = 29889 # "."
            
            if is14:
              labels[i, len(response_token_ids)+response_token_ids_start_idx+1:] = -100
            else:
              labels[i, len(response_token_ids)+response_token_ids_start_idx:] = -100
            
            # set the last label to 2, to mark the end of the input prompt
            labels[i,-1] = 2

        batch["labels"] = labels

        return batch

In [ ]:
# !!! IMPORTANT !!! for icews18-ragtkgc, you already have the dataset of size 1024.
# For all other history modeling algorithms, you will have to manually select them.
# uncomment the following lines for it.

#length_ti = len(tokenized_input)

#step = round(length_ti / 1024)

#tokenized_input = tokenized_input.select(range(0, length_ti, step)[:1024])

In [ ]:
trainer = Trainer(model = training_model, # We pass in the PEFT version of the foundation model or the standard one if full finetuning is desired
                args = training_args, #The args for the training.
                train_dataset = tokenized_input, #The dataset used to to train the model.
                #eval_dataset = vd,
                data_collator=DataCollatorForCompletionLM(tokenizer, mlm=False), # mlm=False indicates not to use masked language modeling
                )

In [ ]:
trainer.train()